In [37]:
'''import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# Generate synthetic data (same as before)
np.random.seed(42)
n_periods = 100
dates = pd.date_range(start="2015-01", periods=n_periods, freq="M")
seasonal_pattern = np.sin(2 * np.pi * np.arange(n_periods) / 12) * 10
trend = np.linspace(10, 20, n_periods)
noise = np.random.normal(0, 2, n_periods)
sales = 50 + seasonal_pattern + trend + noise
advertising_spend = 10 + np.random.normal(0, 1, n_periods)


data = pd.DataFrame({"Sales": sales, "Advertising": advertising_spend}, index=dates)
train_size = int(n_periods * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]
exog_train, exog_test = train[["Advertising"]], test[["Advertising"]]


# Use auto_arima to find the best p, d, q values and seasonal order
# Set seasonal=True and specify the seasonal_period (e.g., 12 for monthly data with yearly seasonality)
auto_model = auto_arima(train["Sales"], exogenous=exog_train, seasonal=True, m=12, trace=True,
                        error_action='ignore', suppress_warnings=True, stepwise=True)


# Get the optimal parameters
order = auto_model.order            # (p, d, q)
seasonal_order = auto_model.seasonal_order  # (P, D, Q, S)


print(f"Optimal order: {order}")
print(f"Optimal seasonal order: {seasonal_order}")


# Define and fit the SARIMAX model with the identified orders
model = SARIMAX(train["Sales"], exog=exog_train, order=order, seasonal_order=seasonal_order)
results = model.fit()


# Print model summary
print(results.summary())


# Forecasting with the SARIMAX model
n_forecast = len(test)
forecast = results.get_forecast(steps=n_forecast, exog=exog_test)


# Extracting forecasted values and confidence intervals
forecast_values = forecast.predicted_mean
conf_int = forecast.conf_int()


# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train["Sales"], label="Training Data")
plt.plot(test.index, test["Sales"], label="Test Data", color="gray")
plt.plot(forecast_values.index, forecast_values, label="Forecast", color="blue")
plt.fill_between(forecast_values.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1],
                 color='blue', alpha=0.2)
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("SARIMAX Forecast with Optimal Parameters")
plt.legend()
plt.show()'''


'import numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom statsmodels.tsa.statespace.sarimax import SARIMAX\nfrom pmdarima import auto_arima\nfrom selenium import webdriver\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.support.ui import WebDriverWait\nfrom selenium.webdriver.support import expected_conditions as EC\nimport time\n\n\n# Generate synthetic data (same as before)\nnp.random.seed(42)\nn_periods = 100\ndates = pd.date_range(start="2015-01", periods=n_periods, freq="M")\nseasonal_pattern = np.sin(2 * np.pi * np.arange(n_periods) / 12) * 10\ntrend = np.linspace(10, 20, n_periods)\nnoise = np.random.normal(0, 2, n_periods)\nsales = 50 + seasonal_pattern + trend + noise\nadvertising_spend = 10 + np.random.normal(0, 1, n_periods)\n\n\ndata = pd.DataFrame({"Sales": sales, "Advertising": advertising_spend}, index=dates)\ntrain_size = int(n_periods * 0.8)\ntrain, test = data.iloc[:train_size], data.iloc[train_size:]\nexog_train, exog_test

In [ ]:
import pandas as pd
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import csv

def read_csv_to_nested_list(file_path):
    nested_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the header row
        # Strip any extra spaces from the headers
        headers = [header.strip() for header in headers]
        
        # Find index of 'date' and 'y' columns
        date_idx = headers.index('date')
        y_idx = headers.index('y')
        
        # Read the remaining rows
        for row in reader:
            date = row[date_idx].strip()
            y_value = float(row[y_idx].strip())
            nested_list.append([date, y_value])
    
    return nested_list

# Example usage
file_path = 'Line Plot.csv'  # Replace with the path to your CSV file
data = read_csv_to_nested_list(file_path)
print(data , len(data))

df_exog = pd.read_csv('exogenous_variables.csv', parse_dates=['Date'])
df_exog.set_index("Date", inplace=True)

# Remove '%' symbol and convert to numeric
for column in df_exog.columns:
    df_exog[column] = df_exog[column].astype(str).str.replace('%', '')  # Remove '%' symbols
    df_exog[column] = pd.to_numeric(df_exog[column], errors='coerce')    # Convert to numeric, set errors='coerce' for NaNs

# Interpolate to handle any NaNs that might be created from conversion

# Step 1: Convert nested list to DataFrame
df = pd.DataFrame(data, columns=["date", "y"])
df["date"] = pd.to_datetime(df["date"])  # Convert date strings to datetime
df.set_index("date", inplace=True)       # Set 'Date' as the DataFrame index

df = df.groupby("date").mean()
df_exog = df_exog.interpolate(method='linear')

# Now set the date as the index

print(df)
print(df_exog.head())


##########################################################################################
# Step 2: Use auto_arima to determine the best order and seasonal_order
# Since the data is weekly and we want quarterly seasonality, set `m=13`
auto_model = auto_arima(df["y"],
                        exogenous=df_exog, 
                        seasonal=True,  
                        trace=True,
                        m = 13,
                        error_action='ignore', 
                        suppress_warnings=True, 
                        stepwise=True)  # Set D = 0

# Retrieve the best p, d, q and seasonal P, D, Q values
order = auto_model.order
seasonal_order = auto_model.seasonal_order

print("Optimal order:", order)
print("Optimal seasonal order:", seasonal_order)

# Step 3: Fit SARIMAX model with the identified parameters
model = SARIMAX(df["y"], order=order, seasonal_order=seasonal_order)
sarimax_model = model.fit(disp=False)

# Step 4: Forecast (e.g., forecast the next 10 weeks)
forecast = sarimax_model.get_forecast(steps=10)
predicted_values = forecast.predicted_mean

print(predicted_values)

/tmp/ipykernel_10224/1842521361.py:40: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.asfreq('Q')
/tmp/ipykernel_10224/1842521361.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_exog['Date'] = pd.to_datetime(df_exog['Date'], errors='coerce')
/tmp/ipykernel_10224/1842521361.py:56: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_exog = df_exog.asfreq('Q').interpolate(method='linear')


ValueError: Input contains NaN.